## Load libraries

In [ ]:
import sys
sys.path.append('/home/ubuntu/hrlcm/hrlcm')

In [ ]:
import argparse
from augmentation import *
from dataset import *
from tqdm.auto import tqdm
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import pickle as pkl
from models.deeplab import DeepLab
from models.unet import UNet
from train import Trainer
from loss import BalancedCrossEntropyLoss
import torch_optimizer as optim

In [ ]:
import os
os.chdir('/home/ubuntu/hrlcm')

## Get dummy inline arguments 

In [ ]:
# Define a dummy args for testing
class args_dummy:
    def __init__(self):
        self.exp_name = 'unet_norm_adabound_compose_lr'
        self.data_dir = 'results/north'
        self.out_dir = 'results/dl'
        self.lowest_score = 10
        self.noise_ratio = 0
        self.trans_prob = 0.5
        self.label_offset = 1
        self.rg_rotate = '-90, 90'
        self.model = 'unet'
        self.train_mode = 'single'
        self.out_stride = 8
        self.gpu_devices = '0, 1, 2, 3'
        self.sync_norm = False
        self.max_lr = 0.001
        self.base_lr = 0.001
        self.clr_gamma = 0.9999
        self.save_freq = 10
        self.log_feq = 10
        self.batch_size = 32
        self.epochs = 200
        self.optimizer_name = 'AdamP'
        self.resume = None
        self.checkpoint_dir = os.path.join(self.out_dir, self.exp_name, 'checkpoints')
        self.logs_dir = os.path.join(self.out_dir, self.exp_name, 'logs')


# Initialize dummy args
args = args_dummy()

In [ ]:
# Set directory for saving files
if args.exp_name:
    args.checkpoint_dir = os.path.join(args.out_dir, args.exp_name, 'checkpoints')
    args.logs_dir = os.path.join(args.out_dir, args.exp_name, 'logs')
else:
    args.checkpoint_dir = os.path.join(args.out_dir, args.model, 'checkpoints')
    args.logs_dir = os.path.join(args.out_dir, args.model, 'logs')

# Create dirs if necessary
if not os.path.isdir(args.checkpoint_dir):
    os.makedirs(args.checkpoint_dir)
if not os.path.isdir(args.logs_dir):
    os.makedirs(args.logs_dir)

# Dir for mean and sd pickles
args.stats_dir = os.path.join(args.data_dir, 'norm_stats')

In [ ]:
# Set flags for GPU processing if available
if torch.cuda.is_available():
    args.use_gpu = True
else:
    args.use_gpu = False

# Load dataset
# Define rotate degrees
args.rg_rotate = tuple(float(each) for each in args.rg_rotate.split(','))

# synchronize transform for train dataset
sync_transform = Compose([
#     RandomScale(prob=args.trans_prob),
    RandomFlip(prob=args.trans_prob),
#     RandomCenterRotate(degree=args.rg_rotate,
#                        prob=args.trans_prob),
    SyncToTensor()
])

# synchronize transform for validate dataset
val_transform = Compose([
    SyncToTensor()
])

# Image transform
# Load mean and sd for normalization
with open(os.path.join(args.stats_dir,
                       "means.pkl"), "rb") as input_file:
    mean = tuple(pkl.load(input_file))

with open(os.path.join(args.stats_dir,
                       "stds.pkl"), "rb") as input_file:
    std = tuple(pkl.load(input_file))
img_transform = ImgNorm(mean, std)

## Load datasets 

In [ ]:
# Get train dataset
train_dataset = NFSEN1LC(data_dir=args.data_dir,
                         usage='train',
                         lowest_score=args.lowest_score,
                         noise_ratio=args.noise_ratio,
                         label_offset=args.label_offset,
                         sync_transform=sync_transform,
                         img_transform=img_transform,
                         label_transform=None)
# Put into DataLoader
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=args.batch_size,
                          num_workers=32,
                          shuffle=True,
                          pin_memory=True,
                          drop_last=True)

# Get validate dataset
validate_dataset = NFSEN1LC(data_dir=args.data_dir,
                            usage='validate',
                            label_offset=args.label_offset,
                            sync_transform=val_transform,
                            img_transform=img_transform,
                            label_transform=None)
# Put into DataLoader
validate_loader = DataLoader(dataset=validate_dataset,
                             batch_size=args.batch_size,
                             num_workers=32,
                             shuffle=False,
                             pin_memory=True,
                             drop_last=False)

## Set model 

In [ ]:
# Set up network
args.n_classes = train_dataset.n_classes
args.n_channels = train_dataset.n_channels
if args.model == "deeplab":
    model = DeepLab(num_classes=args.n_classes,
                    backbone='resnet',
                    pretrained_backbone=False,
                    output_stride=args.out_stride,
                    sync_bn=False,
                    freeze_bn=False,
                    n_in=args.n_channels)
else:
    model = UNet(n_classes=args.n_classes,
                 n_channels=args.n_channels)

args.use_gpu = torch.cuda.is_available()
    # Get devices
if args.gpu_devices:
    args.gpu_devices = [int(each) for each in args.gpu_devices.split(',')]

# Set model
if args.use_gpu:
    if args.gpu_devices:
        torch.cuda.set_device(args.gpu_devices[0])
        model = torch.nn.DataParallel(model, device_ids=args.gpu_devices)
        if args.sync_norm:
            model = convert_model(model)
    model = model.cuda()

# Define loss function
loss_fn = BalancedCrossEntropyLoss()

# Set up tensorboard logging
writer = SummaryWriter(log_dir=args.logs_dir)

# Save config
pkl.dump(args, open(os.path.join(args.checkpoint_dir, "args.pkl"), "wb"))

## Run epochs 

### Constant learning rate 

In [ ]:
# Define optimizer
# if args.optimizer_name == 'Adadelta':
#     optimizer = torch.optim.Adadelta(model.parameters(),
#                                      lr=args.max_lr)
# elif args.optimizer_name == 'Adam':
#     optimizer = torch.optim.Adam(model.parameters(),
#                                  lr=args.max_lr,
#                                  amsgrad=True)
# elif args.optimizer_name == 'AdamW':
#     optimizer = torch.optim.AdamW(model.parameters(),
#                                  lr=args.max_lr,
#                                  amsgrad=True)
# else:
#     print('Not supported optimizer, use Adam instead.')
#     optimizer = torch.optim.Adam(model.parameters(),
#                                  lr=args.max_lr,
#                                  amsgrad=True)
optimizer = optim.AdaBound(model.parameters(), lr=0.001, final_lr=0.01, amsbound=True)
# optimizer = optim.AdamP(model.parameters(), nesterov=True, lr=0.01)

In [ ]:
# With constant learning rate
step = 0
trainer = Trainer(args)
pbar = tqdm(total=args.epochs, desc="[Epoch]")
for epoch in range(args.epochs):
    # Run training for one epoch
    model, step = trainer.train(model, train_loader, loss_fn,
                                optimizer, writer, step=step)
    # Run validation
    trainer.validate(model, validate_loader, step, loss_fn, writer)

    # Save checkpoint
    if epoch % args.save_freq == 0:
        trainer.export_model(model, optimizer=optimizer, step=step)

    # Update pbar
    pbar.set_description("[Epoch] lr: {:.4f}".format(
                round(optimizer.param_groups[0]["lr"], 4)))
    pbar.update()

# Export final set of weights
trainer.export_model(model, optimizer, name="final")

# Close pbar
pbar.close()

### With learning rate scheduler 

#### Single scheduler 

In [ ]:
# Define optimizer
# if args.optimizer_name == 'Adadelta':
#     optimizer = torch.optim.Adadelta(model.parameters(),
#                                      lr=args.max_lr)
# elif args.optimizer_name == 'Adam':
#     optimizer = torch.optim.Adam(model.parameters(),
#                                  lr=args.max_lr,
#                                  amsgrad=True)
# elif args.optimizer_name == 'AdamW':
#     optimizer = torch.optim.AdamW(model.parameters(),
#                                  lr=args.max_lr,
#                                  amsgrad=True)
# else:
#     print('Not supported optimizer, use Adam instead.')
#     optimizer = torch.optim.Adam(model.parameters(),
#                                  lr=args.max_lr,
#                                  amsgrad=True)
# optimizer = optim.AdaBound(model.parameters(), lr=0.001, final_lr=0.01, amsbound=True)
optimizer = optim.AdamP(model.parameters(), nesterov=True, lr=0.01)

In [ ]:
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
step = 0
trainer = Trainer(args)
pbar = tqdm(total=args.epochs, desc="[Epoch]")
for epoch in range(args.epochs):
    # Run training for one epoch
    model, step = trainer.train(model, train_loader, loss_fn,
                                optimizer, writer, step=step)
    # Run validation
    trainer.validate(model, validate_loader, step, loss_fn, writer)
    
    # Update learning rate
    scheduler.step()

    # Save checkpoint
    if epoch % args.save_freq == 0:
        trainer.export_model(model, optimizer=optimizer, step=step)

    # Update pbar
    pbar.set_description("[Epoch] lr: {:.4f}".format(
                round(optimizer.param_groups[0]["lr"], 4)))
    pbar.update()

# Export final set of weights
trainer.export_model(model, optimizer, name="final")

# Close pbar
pbar.close()

#### Composed scheduler 

In [ ]:
# Visualize learning rate curve
import matplotlib.pyplot as plt
lr_scheduler_1 = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50, eta_min=0.001)
lr_scheduler_2 = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.001, max_lr=0.005, 
                                                   step_size_up=1, step_size_down=3,
                                                   gamma=0.988, cycle_momentum=False,
                                                   mode='exp_range')

lrs = []

for i in range(200):
    optimizer.step()
    if i <= lr_scheduler_1.T_max:
        lr_scheduler_1.step()
    else:
        lr_scheduler_2.step()
    lrs.append(
        optimizer.param_groups[0]["lr"]
    )

plt.plot(lrs)

In [ ]:
lr_scheduler_1 = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20, eta_min=0.001)
lr_scheduler_2 = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.001, max_lr=0.05, 
                                                   step_size_up=1, step_size_down=3,
                                                   gamma=0.96, cycle_momentum=False,
                                                   mode='exp_range')
step = 0
trainer = Trainer(args)
pbar = tqdm(total=args.epochs, desc="[Epoch]")
for epoch in range(args.epochs):
    # Run training for one epoch
    model, step = trainer.train(model, train_loader, loss_fn,
                                optimizer, writer, step=step)
    # Run validation
    trainer.validate(model, validate_loader, step, loss_fn, writer)
    
    # Update learning rate
    if epoch <= lr_scheduler_1.T_max:
        lr_scheduler_1.step()
    else:
        lr_scheduler_2.step()

    # Save checkpoint
    if epoch % args.save_freq == 0:
        trainer.export_model(model, optimizer=optimizer, step=step)

    # Update pbar
    pbar.set_description("[Epoch] lr: {:.4f}".format(
                round(optimizer.param_groups[0]["lr"], 4)))
    pbar.update()

# Export final set of weights
trainer.export_model(model, optimizer, name="final")

# Close pbar
pbar.close()

#### Composed optimizer and scheduler 

In [ ]:
# Define optimizer
optimizer = optim.AdaBound(model.parameters(), lr=0.001, final_lr=0.01, amsbound=True)

In [ ]:
# Visualize learning rate curve
import matplotlib.pyplot as plt

lrs = []

for i in range(200):
    optimizer.step()
    if i <= 30:
        if i == 30:
            lr_scheduler_1 = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.0008, max_lr=0.0012, 
                                                   step_size_up=1, step_size_down=3,
                                                   gamma=0.97, cycle_momentum=False,
                                                   mode='exp_range')
    elif i <= 120:
        lr_scheduler_1.step()
        if i == 120:
            lr_scheduler_2 = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.0004, max_lr=0.0006, 
                                                   step_size_up=1, step_size_down=5,
                                                   gamma=0.94, cycle_momentum=False,
                                                   mode='exp_range')
    else:
        lr_scheduler_2.step()
    lrs.append(
        optimizer.param_groups[0]["lr"]
    )

plt.plot(lrs)

In [ ]:
# Define optimizer
optimizer = optim.AdaBound(model.parameters(), lr=0.001, final_lr=0.01, amsbound=True)

step = 0
trainer = Trainer(args)
pbar = tqdm(total=args.epochs, desc="[Epoch]")
for epoch in range(args.epochs):
    # Run training for one epoch
    model, step = trainer.train(model, train_loader, loss_fn,
                                optimizer, writer, step=step)
    # Run validation
    trainer.validate(model, validate_loader, step, loss_fn, writer)
    
    # Update learning rate
    if epoch <= 30:
        if epoch == 30:
            lr_scheduler_1 = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.0008, max_lr=0.0012, 
                                                   step_size_up=1, step_size_down=3,
                                                   gamma=0.97, cycle_momentum=False,
                                                   mode='exp_range')
    elif epoch > 30 and epoch <= 120:
        lr_scheduler_1.step()
        if epoch == 120:
            lr_scheduler_2 = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.0004, max_lr=0.0006, 
                                                   step_size_up=1, step_size_down=5,
                                                   gamma=0.94, cycle_momentum=False,
                                                   mode='exp_range')
    else:
        lr_scheduler_2.step()

    # Save checkpoint
    if epoch % args.save_freq == 0:
        trainer.export_model(model, optimizer=optimizer, step=step)

    # Update pbar
    pbar.set_description("[Epoch] lr: {:.4f}".format(
                round(optimizer.param_groups[0]["lr"], 4)))
    pbar.update()

# Export final set of weights
trainer.export_model(model, optimizer, name="final")

# Close pbar
pbar.close()

## Evaluate 

### Define functions and load packages 

In [ ]:
import argparse
from augmentation import *
from dataset import *
from metrics import ConfMatrix
from torch.utils.data import DataLoader
import pickle as pkl
from models.deeplab import DeepLab
from models.unet import UNet
from tqdm.auto import tqdm
import torch.nn as nn
import numpy as np
from sklearn.metrics import f1_score, precision_score, recall_score, \
    fbeta_score, classification_report, hamming_loss

class Precision_score(nn.Module):

    def __init__(self):
        super().__init__()

    @staticmethod
    def forward(predict_labels, true_labels):
        weighted_prec = precision_score(true_labels, predict_labels, average='weighted')

        return weighted_prec


class Recall_score(nn.Module):

    def __init__(self):
        super().__init__()

    @staticmethod
    def forward(predict_labels, true_labels):
        weighted_rec = recall_score(true_labels, predict_labels, average='weighted')

        return weighted_rec


class F1_score(nn.Module):

    def __init__(self):
        super().__init__()

    @staticmethod
    def forward(predict_labels, true_labels):
        weighted_f1 = f1_score(true_labels, predict_labels, average="weighted")

        return weighted_f1


class F2_score(nn.Module):

    def __init__(self):
        super().__init__()

    @staticmethod
    def forward(predict_labels, true_labels):
        weighted_f2 = fbeta_score(true_labels, predict_labels, beta=2, average="weighted")

        return weighted_f2


class Hamming_loss(nn.Module):

    def __init__(self):
        super().__init__()

    @staticmethod
    def forward(predict_labels, true_labels):
        return hamming_loss(true_labels, predict_labels)


class cls_report(nn.Module):
    def __init__(self, target_names):
        super().__init__()
        self.target_names = target_names

    def forward(self, predict_labels, true_labels):
        report = classification_report(true_labels, predict_labels,
                                       target_names=self.target_names,
                                       output_dict=True)

        return report

In [ ]:
class eval_args_dummy:
    def __init__(self):
        self.args_path = 'results/dl/unet_norm_adabound_compose_lr/checkpoints/args.pkl'
        self.checkpoint_file = 'results/dl/unet_norm_adabound_compose_lr/checkpoints/final.pth'
        self.stats_dir = 'results/north/norm_stats'
        self.data_dir = 'results/north'
        self.out_dir = 'results/evaluation'
        self.label_offset = 1
        self.num_workers = 16
        self.batch_size = 32
        self.gpu_devices = '0, 1, 2, 3'


# Initialize dummy args
args = eval_args_dummy()

In [ ]:
print("=" * 20, "PREDICTION CONFIG", "=" * 20)
for arg in vars(args):
    print('{0:20}  {1}'.format(arg, getattr(args, arg)))
print()

# Load config of training
train_args = pkl.load(open(args.args_path, "rb"))
print("=" * 20, "TRAIN CONFIG", "=" * 20)
for arg in vars(train_args):
    print('{0:20}  {1}'.format(arg, getattr(train_args, arg)))
print()

# Set flags for GPU processing if available
args.use_gpu = torch.cuda.is_available()

# Create output dir
os.makedirs(args.out_dir, exist_ok=True)

# Load dataset
# synchronize transform for validate dataset
val_transform = Compose([
    SyncToTensor()
])

# Image transform
# Load mean and sd for normalization
with open(os.path.join(args.stats_dir,
                       "means.pkl"), "rb") as input_file:
    mean = tuple(pkl.load(input_file))

with open(os.path.join(args.stats_dir,
                       "stds.pkl"), "rb") as input_file:
    std = tuple(pkl.load(input_file))
img_transform = ImgNorm(mean, std)

# Get validate dataset
validate_dataset = NFSEN1LC(data_dir=args.data_dir,
                            usage='validate',
                            label_offset=args.label_offset,
                            sync_transform=val_transform,
                            img_transform=img_transform,
                            label_transform=None)
# Put into DataLoader
validate_loader = DataLoader(dataset=validate_dataset,
                             batch_size=args.batch_size,
                             num_workers=args.num_workers,
                             shuffle=False,
                             drop_last=False)

# set up network
if train_args.model == "deeplab":
    model = DeepLab(num_classes=train_args.n_classes,
                    backbone='resnet',
                    pretrained_backbone=False,
                    output_stride=train_args.out_stride,
                    sync_bn=False,
                    freeze_bn=False,
                    n_in=train_args.n_channels)
else:
    model = UNet(n_classes=train_args.n_classes,
                 n_channels=train_args.n_channels)

# Get devices
if args.gpu_devices:
    args.gpu_devices = [int(each) for each in args.gpu_devices.split(',')]

if args.use_gpu:
    if args.gpu_devices:
        torch.cuda.set_device(args.gpu_devices[0])
        model = torch.nn.DataParallel(model, device_ids=args.gpu_devices)
    model = model.cuda()

# Restore network weights
state = torch.load(args.checkpoint_file)
model.load_state_dict(state["model_state_dict"])
model.eval()
print("Loaded checkpoint")

# predict samples
# define metrics
prec_score_ = Precision_score()
recal_score_ = Recall_score()
f1_score_ = F1_score()
f2_score_ = F2_score()
hamming_loss_ = Hamming_loss()
types = validate_dataset.lc_types
classification_report_ = cls_report(types)

# Prediction
y_true = []
predicted_probs = []
conf_mat = ConfMatrix(validate_loader.dataset.n_classes)
with torch.no_grad():
    for i, (image, labels) in enumerate(tqdm(validate_loader, desc="Evaluate")):
        # Move data to gpu if model is on gpu
        if args.use_gpu:
            image = image.to(torch.device("cuda"))
        print(image.shape)

        # Forward pass
        logits = model(image)

        # Update confusion matrix
        conf_mat.add_batch(labels, logits.max(1)[1])

        # Convert logits to probabilities
        sm = torch.nn.Softmax(dim=1)
        probs = sm(logits).cpu().numpy()

        labels = labels.cpu().numpy()  # keep true & pred label at same loc.
        predicted_probs += list(probs)
        y_true += list(labels)

predicted_probs = np.asarray(predicted_probs)

# Convert predicted probabilities into one-hot labels
y_predicted = np.argmax(predicted_probs, axis=1).flatten()
y_true = np.asarray(y_true).flatten()

# Evaluation with metrics
f1 = f1_score_(y_predicted, y_true)
f2 = f2_score_(y_predicted, y_true)
prec = prec_score_(y_predicted, y_true)
rec = recal_score_(y_predicted, y_true)
hm_loss = hamming_loss_(y_predicted, y_true)
report = classification_report_(y_predicted, y_true)
aa = conf_mat.get_aa()

info = {"weightedPrec": prec,
        "weightedRec": rec,
        "weightedF1": f1,
        "weightedF2": f2,
        "HammingLoss": hm_loss,
        "clsReport": report,
        "conf_mat": conf_mat.norm_on_lines(),
        "AverageAcc": aa}

print("Save out metrics")
pkl.dump(info,open(os.path.join(
                 args.out_dir,"{}_evaluation.pkl"
                     .format(train_args.exp_name)), "wb"))

In [ ]:
info

## Do one prediction 

In [ ]:
import argparse
from augmentation import *
from dataset import *
from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch.nn.functional as F
import pickle as pkl
from models.deeplab import DeepLab
from models.unet import UNet
from tqdm.auto import tqdm

In [ ]:
class pred_args_dummy:
    def __init__(self):
        self.args_path = 'results/dl/unet_norm_adabound_compose_lr/checkpoints/args.pkl'
        self.checkpoint_file = 'results/dl/unet_norm_adabound_compose_lr/checkpoints/final.pth'
        self.stats_dir = 'results/north/norm_stats'
        self.data_dir = 'results/north'
        self.out_dir = 'results/prediction'
        self.label_offset = 1
        self.num_workers = 16
        self.batch_size = 32
        self.gpu_devices = '0, 1, 2, 3'


# Initialize dummy args
args = pred_args_dummy()

In [ ]:
print("=" * 20, "PREDICTION CONFIG", "=" * 20)
for arg in vars(args):
    print('{0:20}  {1}'.format(arg, getattr(args, arg)))
print()

# Load config of training
train_args = pkl.load(open(args.args_path, "rb"))
print("=" * 20, "TRAIN CONFIG", "=" * 20)
for arg in vars(train_args):
    print('{0:20}  {1}'.format(arg, getattr(train_args, arg)))
print()

# Create output dir
os.makedirs(args.out_dir, exist_ok=True)

# Set flags for GPU processing if available
args.use_gpu = torch.cuda.is_available()

# set up network
if train_args.model == "deeplab":
    model = DeepLab(num_classes=train_args.n_classes,
                    backbone='resnet',
                    pretrained_backbone=False,
                    output_stride=train_args.out_stride,
                    sync_bn=False,
                    freeze_bn=False,
                    n_in=train_args.n_channels)
else:
    model = UNet(n_classes=train_args.n_classes,
                 n_channels=train_args.n_channels)
# Get devices
if args.gpu_devices:
    args.gpu_devices = [int(each) for each in args.gpu_devices.split(',')]

if args.use_gpu:
    if args.gpu_devices:
        torch.cuda.set_device(args.gpu_devices[0])
        model = torch.nn.DataParallel(model, device_ids=args.gpu_devices)
    model = model.cuda()

# Restore network weights
state = torch.load(args.checkpoint_file)
model.load_state_dict(state["model_state_dict"])
model.eval()
print("Loaded checkpoint from {}".format(args.checkpoint_file))

# Predict
# Image transform
# Load mean and sd for normalization
with open(os.path.join(args.stats_dir,
                       "means.pkl"), "rb") as input_file:
    mean = tuple(pkl.load(input_file))

with open(os.path.join(args.stats_dir,
                       "stds.pkl"), "rb") as input_file:
    std = tuple(pkl.load(input_file))
pred_transform = ComposeImg([
    ImgToTensor(),
    ImgNorm(mean, std)
])

In [ ]:
catalog = pd.read_csv(os.path.join(args.data_dir, 'dl_catalog_predict.csv'))
tile_id = '1224-997'
# Get validate dataset
predict_dataset = NFSEN1LC(data_dir=args.data_dir,
                           usage='predict',
                           sync_transform=None,
                           img_transform=pred_transform,
                           label_transform=None,
                           tile_id=tile_id)
# Put into DataLoader
predict_loader = DataLoader(dataset=predict_dataset,
                            batch_size=args.batch_size,
                            num_workers=args.num_workers,
                            shuffle=False,
                            drop_last=False)

# Set and crate paths
score_path = os.path.join(args.out_dir, 'score')
class_path = os.path.join(args.out_dir, 'class')
if not os.path.isdir(score_path):
    os.mkdir(score_path)
if not os.path.isdir(class_path):
    os.mkdir(class_path)

# File names
name_score = os.path.join(score_path, 'score_{}'.format(tile_id))
name_class = os.path.join(class_path, 'class_{}.tif'.format(tile_id))

In [ ]:
# Create dummy tile
meta = predict_dataset.meta
n_class = predict_dataset.n_classes
canvas = np.zeros((1, meta['height'], meta['width']),
                  dtype=meta['dtype'])
canvas_score_ls = []

with torch.no_grad():
    for i, (img, index_full) in enumerate(tqdm(predict_loader, desc="Predict")):
        # GPU setting
        if args.use_gpu:
            img = img.cuda()

        out = F.softmax(model(img), 1)
        batch, n_class, width, height = out.size()
        score_width = width
        score_height = height

        # for each batch
        for i in range(batch):
            index = (index_full[0][i], index_full[1][i])
            out_predict = out.max(dim=1)[1][:, :, :].cpu().numpy()[i, :, :]
            out_predict = np.expand_dims(out_predict, axis=0)
            out_predict = out_predict.astype(np.int8)
            canvas[:, index[0]: index[0] + score_width, index[1]: index[1] + score_height] = out_predict

            # scores for each non-background class
            for n in range(n_class):
                out_score = out[:, n, :, :].data[i][:, :].cpu().numpy() * 100
                out_score = np.expand_dims(out_score, axis=0).astype(np.int8)
                try:
                    canvas_score_ls[n][:, index[0]: index[0] + score_width, index[1]: index[1] + score_height] = \
                        out_score
                except:
                    canvas_score_single = np.zeros((1, meta['height'], meta['width']), dtype=meta['dtype'])
                    canvas_score_single[:, index[0]: index[0] + score_width, index[1]: index[1] + score_height] = \
                        out_score
                    canvas_score_ls.append(canvas_score_single)

# Save out
with rasterio.open(name_class, 'w', **meta) as dst:
    dst.write(canvas)

for n in range(n_class):
    with rasterio.open('{}_class{}.tif'.format(name_score, n), 'w', **meta) as dst:
        dst.write(canvas_score_ls[n])
